In [1]:
import cv2
import numpy as np
import json
from PIL import Image

# load the YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# load the class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# load the image
image = cv2.imread('/home/davorin/Documents/OP/horse.jpg')  # replace 'image_path.jpg' with your image path
height, width = image.shape[:2]

# use the YOLO model to perform object detection
blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(net.getUnconnectedOutLayersNames())

# process the results
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# we are only interested in the object with the highest confidence
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
if len(indices) > 0:
    i = indices[0]
    box = boxes[i]
    x, y, w, h = box[0], box[1], box[2], box[3]
    label = str(classes[class_ids[i]])

# create a dict with the detection results
result = {'label': label, 'x': x, 'y': y, 'w': w, 'h': h}

# print the detection results
print(json.dumps(result))


{"label": "horse", "x": 215, "y": 80, "w": 208, "h": 267}
